# Limpieza, Transformacion y Carga final de los datos 

Despues de hacer el Analisis Exploratorio de Datos, llegue a la conclusion que no es necesario eliminar/imputar ningun registro en el df debido a valores atipicos, faltantes y duplicados. Adicionalmente, el unico cambio que se hizo para garantiza la conformidad de los datos fue modificar el tipo de dato de la columna "application_date" de object a datetime64

Teniendo esto en cuenta, el siguiente paso a seguir es realizar las transformaciones necesarias para la carga de los datos. En este punto, la ¡PENDIENTE!

In [ ]:
import pandas as pd
import json
import sys
import os
from sqlalchemy import create_engine
from scripts.db_connector import get_db_engine

In [ ]:
sys.path.append(os.path.abspath(".."))

In [ ]:
conn = get_db_engine()

In [5]:
query = "SELECT * FROM candidates_raw_data;"
df = pd.read_sql(query, conn)

df

C:\Users\valen\AppData\Local\Temp\ipykernel_21100\1512009592.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,1,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,2,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,3,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,4,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,5,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,49997,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,49998,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,49999,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


In [6]:
cols = ["email", "country", "seniority", "technology"]
df[cols] = df[cols].apply(lambda x: x.str.lower())

Al ver el tipo de dato que tiene cada una de las columnas del df la unica columna con un tipo de dato que puede ser cabioado es "application_date". Se procedera a cambiar el tipo de dato de "application_date" de object a datetime

In [7]:
df["application_date"]=df["application_date"].astype("datetime64[ns]")

In [8]:
df.head()

,id,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,1,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,norway,2,intern,data engineer,3,3
1,2,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,panama,10,intern,data engineer,2,10
2,3,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,belarus,4,mid-level,client success,10,9
3,4,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,eritrea,25,trainee,qa manual,7,1
4,5,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,myanmar,13,mid-level,social media community management,9,7


## Transformaciones

Definir si quiero legibilidad en los datos usando hired y not hired o busco mejorar el rendimiento usando 1 y 0

In [9]:
def assign_application_status(df):
    if df["code_challenge_score"] >= 7 and df["technical_interview_score"] >= 7:
        return 1
    else:
        return 0


df["application_status"] = df.apply(assign_application_status, axis=1)

df.head()

,id,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,application_status
0,1,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,norway,2,intern,data engineer,3,3,0
1,2,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,panama,10,intern,data engineer,2,10,0
2,3,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,belarus,4,mid-level,client success,10,9,1
3,4,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,eritrea,25,trainee,qa manual,7,1,0
4,5,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,myanmar,13,mid-level,social media community management,9,7,1


In [10]:
df["application_year"] = df["application_date"].dt.year

In [ ]:
df.drop(columns= ["first_name", "last_name", "email"])

In [12]:
df.head()

,id,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,application_status,application_year
0,1,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,norway,2,intern,data engineer,3,3,0,2021
1,2,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,panama,10,intern,data engineer,2,10,0,2021
2,3,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,belarus,4,mid-level,client success,10,9,1,2020
3,4,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,eritrea,25,trainee,qa manual,7,1,0,2020
4,5,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,myanmar,13,mid-level,social media community management,9,7,1,2020


In [ ]:
with open ("scripts/credentials.json", "r", encoding="utf-8") as file:
    credentials = json.load(file)

db_host = credentials["db_host"]
db_name = credentials["db_name"]
db_user = credentials["db_user"]
db_password = credentials["db_password"]

pg_engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}")

#load data
df.to_sql('candidates_final_data', pg_engine)